In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize

In [2]:
CLIENT_ID = 'C4PCOYEM3A2BGCRL2VLN3Q0FC5D5BMJON3QQTHC3N1VMJSMD' # your Foursquare ID
CLIENT_SECRET = 'LXWRZADBRGIEVUY2PFRYK5DRUGBP0CV2KIGECR3EU1ZMNIJK' # your Foursquare Secret
VERSION = '20180605'
LIMIT = 30

In [3]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent = 'Lucas-Toronto')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.653963 -79.387207


In [24]:
category = '4bf58dd8d48988d118951735'
radius = 5000

In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, category)
url

'https://api.foursquare.com/v2/venues/search?client_id=C4PCOYEM3A2BGCRL2VLN3Q0FC5D5BMJON3QQTHC3N1VMJSMD&client_secret=LXWRZADBRGIEVUY2PFRYK5DRUGBP0CV2KIGECR3EU1ZMNIJK&ll=43.653963,-79.387207&v=20180605&radius=5000&limit=30&categoryId=4bf58dd8d48988d118951735'

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d275d9e342adf0038c52cb0'},
 'response': {'venues': [{'id': '4adb87b0f964a520ad2821e3',
    'name': '金城超級市場 Lucky Moose Food Mart',
    'location': {'address': '393 Dundas St W',
     'crossStreet': 'at Beverly St',
     'lat': 43.65365148008927,
     'lng': -79.3944133930438,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.65365148008927,
       'lng': -79.3944133930438}],
     'distance': 581,
     'postalCode': 'M5T 1G6',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['393 Dundas St W (at Beverly St)',
      'Toronto ON M5T 1G6',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d118951735',
      'name': 'Grocery Store',
      'pluralName': 'Grocery Stores',
      'shortName': 'Grocery Store',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-15628609

In [25]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",False,4adb87b0f964a520ad2821e3,393 Dundas St W,CA,Toronto,Canada,at Beverly St,581,"[393 Dundas St W (at Beverly St), Toronto ON M...","[{'label': 'display', 'lat': 43.65365148008927...",43.653651,-79.394413,NaN,M5T 1G6,ON,金城超級市場 Lucky Moose Food Mart,v-1562860959,NaN
1,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",False,5c3c8a9dccad6b00390ec282,1 Bloor St E,CA,Toronto,Canada,NaN,1755,"[1 Bloor St E, Toronto ON M4W 0A8, Canada]","[{'label': 'display', 'lat': 43.66972962086999...",43.669730,-79.386872,NaN,M4W 0A8,ON,McEwan,v-1562860959,NaN
2,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",False,4ad4c063f964a52052f820e3,87 Avenue Road,CA,Toronto,Canada,Davenport Rd,2100,"[87 Avenue Road (Davenport Rd), Toronto ON M5R...","[{'label': 'display', 'lat': 43.67200542931751...",43.672005,-79.394832,NaN,M5R 3R9,ON,Whole Foods Market,v-1562860959,NaN
3,"[{'id': '52f2ab2ebcbc57f1066b8b46', 'name': 'S...",False,4ad8f18ef964a520401621e3,222 Cherry St.,CA,Toronto,Canada,btwn Commissioners St. & Polson St.,3101,[222 Cherry St. (btwn Commissioners St. & Pols...,"[{'label': 'display', 'lat': 43.64349082636327...",43.643491,-79.351526,NaN,M5A 3L2,ON,T&T Supermarket 大統華超級市場,v-1562860959,NaN
4,"[{'id': '52f2ab2ebcbc57f1066b8b46', 'name': 'S...",False,4af457d8f964a5207ef121e3,100 Lynn Williams St.,CA,Toronto,Canada,Liberty Village,2943,"[100 Lynn Williams St. (Liberty Village), Toro...","[{'label': 'display', 'lat': 43.63941018975886...",43.639410,-79.417722,NaN,M6K 3N6,ON,Metro,v-1562860959,NaN


In [8]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,金城超級市場 Lucky Moose Food Mart,Grocery Store,393 Dundas St W,CA,Toronto,Canada,at Beverly St,581,"[393 Dundas St W (at Beverly St), Toronto ON M...","[{'label': 'display', 'lat': 43.65365148008927...",43.653651,-79.394413,NaN,M5T 1G6,ON,4adb87b0f964a520ad2821e3
1,McEwan,Grocery Store,1 Bloor St E,CA,Toronto,Canada,NaN,1755,"[1 Bloor St E, Toronto ON M4W 0A8, Canada]","[{'label': 'display', 'lat': 43.66972962086999...",43.669730,-79.386872,NaN,M4W 0A8,ON,5c3c8a9dccad6b00390ec282
2,Whole Foods Market,Grocery Store,87 Avenue Road,CA,Toronto,Canada,Davenport Rd,2100,"[87 Avenue Road (Davenport Rd), Toronto ON M5R...","[{'label': 'display', 'lat': 43.67200542931751...",43.672005,-79.394832,NaN,M5R 3R9,ON,4ad4c063f964a52052f820e3
3,T&T Supermarket 大統華超級市場,Supermarket,222 Cherry St.,CA,Toronto,Canada,btwn Commissioners St. & Polson St.,3101,[222 Cherry St. (btwn Commissioners St. & Pols...,"[{'label': 'display', 'lat': 43.64349082636327...",43.643491,-79.351526,NaN,M5A 3L2,ON,4ad8f18ef964a520401621e3
4,Metro,Supermarket,100 Lynn Williams St.,CA,Toronto,Canada,Liberty Village,2943,"[100 Lynn Williams St. (Liberty Village), Toro...","[{'label': 'display', 'lat': 43.63941018975886...",43.639410,-79.417722,NaN,M6K 3N6,ON,4af457d8f964a5207ef121e3
5,Metro,Supermarket,735 College St.,CA,Toronto,Canada,Crawford St.,2589,"[735 College St. (Crawford St.), Toronto ON M6...","[{'label': 'display', 'lat': 43.65471477276098...",43.654715,-79.419337,NaN,M6G 1C5,ON,4aeb1b18f964a5206abe21e3
6,Tim & Sue's No Frills,Grocery Store,372 Pacific Ave,CA,Toronto,Canada,at Dundas St W,6665,"[372 Pacific Ave (at Dundas St W), Toronto ON ...","[{'label': 'display', 'lat': 43.66449894808741...",43.664499,-79.468678,NaN,M6P 2R1,ON,4b6f41b2f964a52074e72ce3
7,Loblaws,Grocery Store,17 Leslie St,CA,Toronto,Canada,at Lakeshore Blvd.,4797,"[17 Leslie St (at Lakeshore Blvd.), Toronto ON...","[{'label': 'display', 'lat': 43.66054929495237...",43.660549,-79.328339,NaN,M4M 3H9,ON,4ad8ebb8f964a520fe1521e3
8,Market 63 By Coppa’s,Grocery Store,1 York St,CA,Toronto,Canada,NaN,1498,"[1 York St, Toronto ON M5J 0B6, Canada]","[{'label': 'display', 'lat': 43.641494, 'lng':...",43.641494,-79.380203,NaN,M5J 0B6,ON,5c76b924d807ee002ce9a33e
9,No Frills,Grocery Store,900 Dufferin St,CA,Toronto,Canada,at Dufferin Mall,3773,"[900 Dufferin St (at Dufferin Mall), Toronto O...","[{'label': 'display', 'lat': 43.65556963168218...",43.655570,-79.434003,NaN,M6H 4A9,ON,4afcaae6f964a520ec2422e3


In [9]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)

folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Center of Toronto',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

for lat, lng, label, in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [16]:
for id in dataframe_filtered.id:
    venue_id = id
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result = requests.get(url).json()
    try:
        print(result['response']['venue']['rating'])
    except:
        print('This venue has not been rated yet.')

7.7
This venue has not been rated yet.
8.7
7.5
6.8
5.2
7.2
6.6
This venue has not been rated yet.
6.3
7.7
7.3
5.7
5.7
7.2
5.1
6.8
5.5
7.9
This venue has not been rated yet.
5.7
7.3
8.4
5.8
7.1
9.1
8.8
9.0
6.2
5.3


In [17]:
ratings_list = ["7.7", "8.7", "7.7", "5.8", "6.6", "7.2", "5.0", "5.2", "6.8", "6.3", "6.4", "7.6" ,"5.7" ,"6.9", "7.2", "7.7", "5.7", "7.0", "8.4", "5.6", "7.1", "6.5", "8.8", "7.3", "7.8", "5.4", "9.1", "6.3", "9.1", "5.8"]
ratings_list

['7.7',
 '8.7',
 '7.7',
 '5.8',
 '6.6',
 '7.2',
 '5.0',
 '5.2',
 '6.8',
 '6.3',
 '6.4',
 '7.6',
 '5.7',
 '6.9',
 '7.2',
 '7.7',
 '5.7',
 '7.0',
 '8.4',
 '5.6',
 '7.1',
 '6.5',
 '8.8',
 '7.3',
 '7.8',
 '5.4',
 '9.1',
 '6.3',
 '9.1',
 '5.8']

In [18]:
#for data in results:
venue = dataframe_filtered.name
rating = ratings_list
df_venueRating = pd.DataFrame(
    {'Venue': venue,
     'Rating':rating})
df_venueRating

,Venue,Rating
0,金城超級市場 Lucky Moose Food Mart,7.7
1,McEwan,8.7
2,Whole Foods Market,7.7
3,T&T Supermarket 大統華超級市場,5.8
4,Metro,6.6
5,Metro,7.2
6,Tim & Sue's No Frills,5.0
7,Loblaws,5.2
8,Market 63 By Coppa’s,6.8
9,No Frills,6.3


In [28]:
#Cluster based off store ratings
k = 3
df_cluster = df_venueRating.drop(['Venue'], 1)
kmeans = KMeans (n_clusters = k, random_state = 0).fit(df_cluster)
kmeans.labels_[0:10]
mm_merged = df_venueRating
mm_merged.insert(0, 'Cluster Labels', kmeans.labels_)
mm_merged = mm_merged.join(df_venueRating.drop(['Rating', 'Cluster Labels'], 1).set_index('Venue'), on = 'Venue')
mm_merged

,Cluster Labels,Venue,Rating
0,2,金城超級市場 Lucky Moose Food Mart,7.7
1,1,McEwan,8.7
2,2,Whole Foods Market,7.7
3,0,T&T Supermarket 大統華超級市場,5.8
4,2,Metro,6.6
4,2,Metro,6.6
4,2,Metro,6.6
4,2,Metro,6.6
5,2,Metro,7.2
5,2,Metro,7.2


In [30]:
map_clusters = folium.Map(location = [latitude, longitude], zoom_start = 11)
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(dataframe_filtered['lat'], dataframe_filtered['lng'], mm_merged['Venue'], mm_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html = True)
    folium.CircleMarker(
        [lat, lon],
    radius = 5,
    popup = label,
    color = rainbow[cluster-1],
    full = True,
    fill_color = rainbow[cluster-1],
    fill_opacity = 0.7).add_to(map_clusters)
map_clusters